In [1]:
import pandas as pd
from sklearn.utils import all_estimators
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import warnings
warnings.simplefilter(action='ignore')

out_file ='full_with_headlines.csv'
head_split = '    '
news_df = pd.read_csv(out_file)
news_df = news_df[(news_df['headlines'].notna()) & (news_df['headlines'] != "")].reset_index()
news_df['winner'] = news_df.groupby(['year','state','district','totalvotes'], sort=False)['candidatevotes'].transform(max)
news_df['winner'] = news_df['winner'] == news_df['candidatevotes']

In [39]:
X = [[x] for x in list(news_df['headlines'].str.split(head_split).str.len())]
y = news_df['winner']

In [40]:
SK_ALL = all_estimators(type_filter=['classifier'],)#'regressor'
names = [sk[0] for sk in SK_ALL if 'dummy' not in sk[0]]
modls = [sk[1] for sk in SK_ALL if 'dummy' not in sk[0]]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True, stratify=y)

In [44]:
res_book = []
for i in range(len(modls)):
    name = names[i]
    try:
        model = modls[i]()
        model.fit(X=X_train, y=y_train)
        preds = model.predict(X_test)
        res_book.append((name,model.score(X_test, y_test),model))
        print((name,model.score(X_test, y_test)))
    except Exception as e:
        #print(name, 'FAIL', str(e)[:100])
        pass

('AdaBoostClassifier', 0.5950738916256157)
('BaggingClassifier', 0.5674876847290641)
('BernoulliNB', 0.5566502463054187)
('CalibratedClassifierCV', 0.5566502463054187)
('ComplementNB', 0.4433497536945813)
('DecisionTreeClassifier', 0.5596059113300492)
('DummyClassifier', 0.5566502463054187)
('ExtraTreeClassifier', 0.5635467980295567)
('ExtraTreesClassifier', 0.5635467980295567)
('GaussianNB', 0.5635467980295567)
('GaussianProcessClassifier', 0.5783251231527093)
('GradientBoostingClassifier', 0.594088669950739)
('HistGradientBoostingClassifier', 0.5802955665024631)
('KNeighborsClassifier', 0.5556650246305419)
('LabelPropagation', 0.5674876847290641)
('LabelSpreading', 0.5704433497536946)
('LinearDiscriminantAnalysis', 0.5566502463054187)
('LinearSVC', 0.5773399014778325)
('LogisticRegression', 0.5566502463054187)
('LogisticRegressionCV', 0.5566502463054187)
('MLPClassifier', 0.5862068965517241)
('MultinomialNB', 0.5566502463054187)
('NearestCentroid', 0.5103448275862069)
('NuSVC', 0.492

In [45]:
res_book2 = [r for r in res_book if r[1] >= 0.57]
print(*res_book2, sep='\n')

('AdaBoostClassifier', 0.5950738916256157, AdaBoostClassifier())
('GaussianProcessClassifier', 0.5783251231527093, GaussianProcessClassifier())
('GradientBoostingClassifier', 0.594088669950739, GradientBoostingClassifier())
('HistGradientBoostingClassifier', 0.5802955665024631, HistGradientBoostingClassifier())
('LabelSpreading', 0.5704433497536946, LabelSpreading())
('LinearSVC', 0.5773399014778325, LinearSVC())
('MLPClassifier', 0.5862068965517241, MLPClassifier())
('SVC', 0.5763546798029556, SVC())


In [46]:
def test_best(m):
    preds = m.predict(X_test)
    print(classification_report(preds,y_test))
test_best(res_book2[0][2])

              precision    recall  f1-score   support

       False       0.56      0.54      0.55       463
        True       0.62      0.64      0.63       552

    accuracy                           0.60      1015
   macro avg       0.59      0.59      0.59      1015
weighted avg       0.59      0.60      0.59      1015



In [29]:
y.sum()/ len(y)

0.5567873860556788